# European Parliament Voted Resolutions

In [2]:
!pip install fuzzywuzzy
import sys
sys.path.append("..")

In [3]:
!pip install python-docx 

In [76]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from fuzzywuzzy import fuzz
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import os
import requests
from docx import Document

from src.data_extraction import *
from src.dataset import DatasetGenerator
from pathlib import Path

pd.set_option('max_colwidth', 200)

In [56]:
input_folder = '../data/raw/'
output_folder = '../data/processed/'

_ = DatasetGenerator(input_folder, output_folder)

In [9]:
_.df

,document_identifier,document_title,document_type,document_parliamentary_term,document_date,document_public_register_notation,document_language,document_pdf,document_doc,document_corrigenda_pdf,document_corrigenda_doc,document_subject_matter,document_directory_code,document_adopts,document_procedure,document_ep_number,document_URI
0,TA-10-2024-0006,Objection to an implementing act: Maximum residue levels for carbendazim and thiophanate‐methyl,EP text adopted,10,2024-09-18,P10_TA(2024)0006,English,NaN,https://data.europarl.europa.eu/distribution/doc/TA-10-2024-0006_en.docx,NaN,NaN,http://publications.europa.eu/resource/authority/subject-matter/PHYT; http://publications.europa.eu/resource/authority/subject-matter/SANT,http://publications.europa.eu/resource/authority/dir-eu-legal-act/035020; http://publications.europa.eu/resource/authority/dir-eu-legal-act/1530,https://data.europarl.europa.eu/eli/dl/doc/B-10-2024-0020,https://data.europarl.europa.eu/eli/dl/proc/2024-2758,PE763.090,https://data.europarl.europa.eu/eli/dl/doc/TA-10-2024-0006
1,TA-10-2024-0005,Numerical strength of interparliamentary delegations,EP text adopted,10,2024-09-18,P10_TA(2024)0005,English,NaN,https://data.europarl.europa.eu/distribution/doc/TA-10-2024-0005_en.docx,NaN,NaN,http://publications.europa.eu/resource/authority/subject-matter/INST,http://publications.europa.eu/resource/authority/dir-eu-legal-act/014020,https://data.europarl.europa.eu/eli/dl/doc/B-10-2024-0019,https://data.europarl.europa.eu/eli/dl/proc/2024-2814,PE762.605,https://data.europarl.europa.eu/eli/dl/doc/TA-10-2024-0005
2,TA-10-2024-0009,The severe situation of political prisoners in Belarus,EP text adopted,10,2024-09-19,P10_TA(2024)0009,English,NaN,https://data.europarl.europa.eu/distribution/doc/TA-10-2024-0009_en.docx,NaN,NaN,http://publications.europa.eu/resource/authority/subject-matter/PESC; http://publications.europa.eu/resource/authority/subject-matter/DDLH,http://publications.europa.eu/resource/authority/dir-eu-legal-act/18,https://data.europarl.europa.eu/eli/dl/doc/B-10-2024-0026; https://data.europarl.europa.eu/eli/dl/doc/B-10-2024-0053; https://data.europarl.europa.eu/eli/dl/doc/B-10-2024-0055; https://data.europa...,https://data.europarl.europa.eu/eli/dl/proc/2024-2804,PE762.610,https://data.europarl.europa.eu/eli/dl/doc/TA-10-2024-0009
3,TA-10-2024-0029,"Situation in Azerbaijan, violation of human rights and international law and relations with Armenia",EP text adopted,10,2024-10-24,P10_TA(2024)0029,English,NaN,https://data.europarl.europa.eu/distribution/doc/TA-10-2024-0029_en.docx,NaN,NaN,http://publications.europa.eu/resource/authority/subject-matter/PESC; http://publications.europa.eu/resource/authority/subject-matter/EXT; http://publications.europa.eu/resource/authority/subject-...,http://publications.europa.eu/resource/authority/dir-eu-legal-act/18; http://publications.europa.eu/resource/authority/dir-eu-legal-act/11401040,https://data.europarl.europa.eu/eli/dl/doc/B-10-2024-0139; https://data.europarl.europa.eu/eli/dl/doc/B-10-2024-0142; https://data.europarl.europa.eu/eli/dl/doc/B-10-2024-0133; https://data.europa...,https://data.europarl.europa.eu/eli/dl/proc/2024-2890,PE764.143,https://data.europarl.europa.eu/eli/dl/doc/TA-10-2024-0029
4,TA-10-2024-0032,Electronic value added tax exemption certificate,EP text adopted,10,2024-11-14,P10_TA(2024)0032,English,NaN,https://data.europarl.europa.eu/distribution/doc/TA-10-2024-0032_en.docx,NaN,NaN,http://publications.europa.eu/resource/authority/subject-matter/FISC; http://publications.europa.eu/resource/authority/subject-matter/TVA; http://publications.europa.eu/resource/authority/subject-...,http://publications.europa.eu/resource/authority/dir-eu-legal-act/093010,https://data.europarl.europa.eu/eli/dl/doc/A-10-2024-0012,https://data.europarl.europa.eu/eli/dl/proc/2024-0152,PE763.197,https://data.europarl.europa.eu/eli/dl/doc/TA-10-2024-0032
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
902,TA-9-

In [10]:
folder = '../data/raw/'
ds_1 = pd.read_csv(folder + "https___data.europarl.europa.eu_distribution_adopted-texts_2023_55_en.csv")
ds_2 = pd.read_csv(folder + "https___data.europarl.europa.eu_distribution_adopted-texts_2024_25_en.csv")

ds = pd.concat([ds_1, ds_2])

In [11]:
read_and_concatenate_csv(folder)

,document_identifier,document_title,document_type,document_parliamentary_term,document_date,document_public_register_notation,document_language,document_pdf,document_doc,document_corrigenda_pdf,document_corrigenda_doc,document_subject_matter,document_directory_code,document_adopts,document_procedure,document_ep_number,document_URI
0,TA-10-2024-0006,Objection to an implementing act: Maximum residue levels for carbendazim and thiophanate‐methyl,EP text adopted,10,2024-09-18,P10_TA(2024)0006,English,NaN,https://data.europarl.europa.eu/distribution/doc/TA-10-2024-0006_en.docx,NaN,NaN,http://publications.europa.eu/resource/authority/subject-matter/PHYT; http://publications.europa.eu/resource/authority/subject-matter/SANT,http://publications.europa.eu/resource/authority/dir-eu-legal-act/035020; http://publications.europa.eu/resource/authority/dir-eu-legal-act/1530,https://data.europarl.europa.eu/eli/dl/doc/B-10-2024-0020,https://data.europarl.europa.eu/eli/dl/proc/2024-2758,PE763.090,https://data.europarl.europa.eu/eli/dl/doc/TA-10-2024-0006
1,TA-10-2024-0005,Numerical strength of interparliamentary delegations,EP text adopted,10,2024-09-18,P10_TA(2024)0005,English,NaN,https://data.europarl.europa.eu/distribution/doc/TA-10-2024-0005_en.docx,NaN,NaN,http://publications.europa.eu/resource/authority/subject-matter/INST,http://publications.europa.eu/resource/authority/dir-eu-legal-act/014020,https://data.europarl.europa.eu/eli/dl/doc/B-10-2024-0019,https://data.europarl.europa.eu/eli/dl/proc/2024-2814,PE762.605,https://data.europarl.europa.eu/eli/dl/doc/TA-10-2024-0005
2,TA-10-2024-0009,The severe situation of political prisoners in Belarus,EP text adopted,10,2024-09-19,P10_TA(2024)0009,English,NaN,https://data.europarl.europa.eu/distribution/doc/TA-10-2024-0009_en.docx,NaN,NaN,http://publications.europa.eu/resource/authority/subject-matter/PESC; http://publications.europa.eu/resource/authority/subject-matter/DDLH,http://publications.europa.eu/resource/authority/dir-eu-legal-act/18,https://data.europarl.europa.eu/eli/dl/doc/B-10-2024-0026; https://data.europarl.europa.eu/eli/dl/doc/B-10-2024-0053; https://data.europarl.europa.eu/eli/dl/doc/B-10-2024-0055; https://data.europa...,https://data.europarl.europa.eu/eli/dl/proc/2024-2804,PE762.610,https://data.europarl.europa.eu/eli/dl/doc/TA-10-2024-0009
3,TA-10-2024-0029,"Situation in Azerbaijan, violation of human rights and international law and relations with Armenia",EP text adopted,10,2024-10-24,P10_TA(2024)0029,English,NaN,https://data.europarl.europa.eu/distribution/doc/TA-10-2024-0029_en.docx,NaN,NaN,http://publications.europa.eu/resource/authority/subject-matter/PESC; http://publications.europa.eu/resource/authority/subject-matter/EXT; http://publications.europa.eu/resource/authority/subject-...,http://publications.europa.eu/resource/authority/dir-eu-legal-act/18; http://publications.europa.eu/resource/authority/dir-eu-legal-act/11401040,https://data.europarl.europa.eu/eli/dl/doc/B-10-2024-0139; https://data.europarl.europa.eu/eli/dl/doc/B-10-2024-0142; https://data.europarl.europa.eu/eli/dl/doc/B-10-2024-0133; https://data.europa...,https://data.europarl.europa.eu/eli/dl/proc/2024-2890,PE764.143,https://data.europarl.europa.eu/eli/dl/doc/TA-10-2024-0029
4,TA-10-2024-0032,Electronic value added tax exemption certificate,EP text adopted,10,2024-11-14,P10_TA(2024)0032,English,NaN,https://data.europarl.europa.eu/distribution/doc/TA-10-2024-0032_en.docx,NaN,NaN,http://publications.europa.eu/resource/authority/subject-matter/FISC; http://publications.europa.eu/resource/authority/subject-matter/TVA; http://publications.europa.eu/resource/authority/subject-...,http://publications.europa.eu/resource/authority/dir-eu-legal-act/093010,https://data.europarl.europa.eu/eli/dl/doc/A-10-2024-0012,https://data.europarl.europa.eu/eli/dl/proc/2024-0152,PE763.197,https://data.europarl.europa.eu/eli/dl/doc/TA-10-2024-0032
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
902,TA-9-

In [12]:
def check_similar_words(df, column, keywords, threshold=80):
    """
    Checks if strings in a pandas column contain words similar to specified keywords.

    Args:
        df (pd.DataFrame): The DataFrame containing the column to check.
        column (str): The name of the column to search for similar words.
        keywords (list of str): A list of target keywords to match against.
        threshold (int): The minimum similarity score to consider a match (default is 80).

    Returns:
        pd.Series: A boolean Series indicating rows with similar matches.
    """
    def has_similar_word(text):
        if pd.isna(text):  # Handle missing values
            return False
        for word in text.split():
            for keyword in keywords:
                if fuzz.ratio(word.lower(), keyword.lower()) >= threshold:
                    return True
        return False

    return df[column].apply(has_similar_word)


In [62]:
keywords = ['ukraine', 'russia', 'israel', 'palestine', 'gaza',
        'conflict', 'war', 'peace', 'occupation',
        'territorial', 'settlement', 'genocide']

In [63]:
check_similar_words(ds, column='document_title', keywords=keywords).value_counts()

document_title
False    860
True      47
Name: count, dtype: int64

In [64]:
ds_filtered = ds[check_similar_words(ds, column='document_title', keywords=keywords)]

In [16]:
def preprocess_text(text):
    """
    Preprocess text by converting to lowercase and removing special characters.

    Args:
        text (str): Input text to preprocess

    Returns:
        str: Preprocessed text
    """
    if pd.isna(text):
        return ''
    # Convert to lowercase
    text = text.lower()
    # Remove special characters and extra whitespaces
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    # Remove extra whitespaces
    text = ' '.join(text.split())
    return text

In [17]:
# Apply text preprocessing to document title
ds['processed_title'] = ds['document_title'].apply(preprocess_text)

In [18]:
ds.head()

,document_identifier,document_title,document_type,document_parliamentary_term,document_date,document_public_register_notation,document_language,document_pdf,document_doc,document_corrigenda_pdf,document_corrigenda_doc,document_subject_matter,document_directory_code,document_adopts,document_procedure,document_ep_number,document_URI,processed_title
0,TA-9-2023-0091,Strengthening the application of the principle of equal pay for equal work or work of equal value between men and women,EP text adopted,9,2023-03-30,P9_TA(2023)0091,English,https://data.europarl.europa.eu/distribution/reds_iPlTa_Itm/TA-9-2023-0091/TA-9-2023-0091-FNL_en.pdf,https://data.europarl.europa.eu/distribution/reds_iPlTa_Itm/TA-9-2023-0091/TA-9-2023-0091-FNL_en.docx,NaN,NaN,http://publications.europa.eu/resource/authority/subject-matter/SOCI; http://publications.europa.eu/resource/authority/subject-matter/DISC; http://publications.europa.eu/resource/authority/subject...,http://publications.europa.eu/resource/authority/dir-eu-legal-act/05202020; http://publications.europa.eu/resource/authority/dir-eu-legal-act/05200520,https://data.europarl.europa.eu/eli/dl/doc/A-9-2022-0056,https://data.europarl.europa.eu/eli/dl/proc/2021-0050,PE693.798,https://data.europarl.europa.eu/eli/dl/doc/TA-9-2023-0091,strengthening the application of the principle of equal pay for equal work or work of equal value between men and women
1,TA-9-2023-0046,Situation of the former President of Georgia Mikheil Saakashvili,EP text adopted,9,2023-02-15,P9_TA(2023)0046,English,https://data.europarl.europa.eu/distribution/reds_iPlTa_Itm/TA-9-2023-0046/TA-9-2023-0046-FNL_en.pdf,https://data.europarl.europa.eu/distribution/reds_iPlTa_Itm/TA-9-2023-0046/TA-9-2023-0046-FNL_en.docx,NaN,NaN,http://publications.europa.eu/resource/authority/subject-matter/PESC; http://publications.europa.eu/resource/authority/subject-matter/DDLH,http://publications.europa.eu/resource/authority/dir-eu-legal-act/18,https://data.europarl.europa.eu/eli/dl/doc/B-9-2023-0109; https://data.europarl.europa.eu/eli/dl/doc/B-9-2023-0106; https://data.europarl.europa.eu/eli/dl/doc/B-9-2023-0112; https://data.europarl....,https://data.europarl.europa.eu/eli/dl/proc/2023-2543,PE741.435,https://data.europarl.europa.eu/eli/dl/doc/TA-9-2023-0046,situation of the former president of georgia mikheil saakashvili
2,TA-9-2023-0037,Electoral rights of mobile Union citizens in European Parliament elections,EP text adopted,9,2023-02-14,P9_TA(2023)0037,English,https://data.europarl.europa.eu/distribution/reds_iPlTa_Itm/TA-9-2023-0037/TA-9-2023-0037-FNL_en.pdf,https://data.europarl.europa.eu/distribution/reds_iPlTa_Itm/TA-9-2023-0037/TA-9-2023-0037-FNL_en.docx,NaN,NaN,http://publications.europa.eu/resource/authority/subject-matter/CIT; http://publications.europa.eu/resource/authority/subject-matter/PRIN,http://publications.europa.eu/resource/authority/dir-eu-legal-act/2020,https://data.europarl.europa.eu/eli/dl/doc/A-9-2022-0297,https://data.europarl.europa.eu/eli/dl/proc/2021-0372,PE704.775,https://data.europarl.europa.eu/eli/dl/doc/TA-9-2023-0037,electoral rights of mobile union citizens in european parliament elections
3,TA-9-2023-0003,Shipments of waste,EP text adopted,9,2023-01-17,P9_TA(2023)0003,English,https://data.europarl.europa.eu/distribution/reds_iPlTa_Itm/TA-9-2023-0003/TA-9-2023-0003-FNL_en.pdf,https://data.europarl.europa.eu/distribution/reds_iPlTa_Itm/TA-9-2023-0003/TA-9-2023-0003-FNL_en.docx,NaN,NaN,http://publications.europa.eu/resource/authority/subject-matter/DECH,http://publications.europa.eu/resource/authority/dir-eu-legal-act/151030,https://data.europarl.europa.eu/eli/dl/doc/A-9-2022-0290,https://data.europarl.europa.eu/eli/dl/proc/2021-0367,PE719.861,https://data.europarl.europa.eu/eli/dl/doc/TA-9-2023-0003,shipments of waste
4,TA-9-2023-0081,EU-Armenia relations,EP text adopted,9,2023-03-15,P9_TA(2023)0081,English,https://data.europarl.europa.eu/distribution/reds_iPlTa_Itm/TA-9-2023-0081/TA-9-2023-0081-FNL_en.pdf,https://

In [24]:
def extract_conflict_keywords(df, column='processed_title'):
    """
    Extract and count keywords related to the Israeli-Palestinian conflict.

    Args:
        df (pd.DataFrame): DataFrame containing the text column
        column (str): Name of the column to analyze

    Returns:
        pd.Series: Counts of conflict-related keywords
    """
    conflict_keywords = [
        'ukraine', 'russia', 'israel', 'palestine', 'gaza',
        'conflict', 'war', 'peace', 'occupation',
        'territorial', 'settlement', 'genocide'
    ]

    # Create a TF-IDF vectorizer
    vectorizer = TfidfVectorizer(vocabulary=conflict_keywords)

    # Fit and transform the text
    tfidf_matrix = vectorizer.fit_transform(df[column])

    # Convert to DataFrame for easier analysis
    tfidf_df = pd.DataFrame(
        tfidf_matrix.toarray(),
        columns=vectorizer.get_feature_names_out(),
        index=df.index
    )

    return tfidf_df

In [25]:
# Extract TF-IDF for conflict-related keywords
conflict_tfidf = extract_conflict_keywords(ds)

In [26]:
# Function to identify documents most relevant to the conflict
def get_conflict_relevant_docs(tfidf_df, threshold=0.1):
    """
    Identify documents most relevant to the conflict based on TF-IDF scores.

    Args:
        tfidf_df (pd.DataFrame): TF-IDF matrix of keywords
        threshold (float): Minimum TF-IDF score to consider a document relevant

    Returns:
        pd.DataFrame: Subset of original dataframe with conflict-relevant documents
    """
    # Sum TF-IDF scores across conflict keywords
    relevance_scores = tfidf_df.sum(axis=1)

    # Filter documents based on relevance threshold
    conflict_docs = ds[relevance_scores >= threshold]

    return conflict_docs

In [27]:
# Get conflict-relevant documents
conflict_relevant_documents = get_conflict_relevant_docs(conflict_tfidf)

In [28]:
conflict_relevant_documents.head()

,document_identifier,document_title,document_type,document_parliamentary_term,document_date,document_public_register_notation,document_language,document_pdf,document_doc,document_corrigenda_pdf,document_corrigenda_doc,document_subject_matter,document_directory_code,document_adopts,document_procedure,document_ep_number,document_URI,processed_title
11,TA-9-2023-0051,The recent deterioration of the inhuman imprisonment conditions of Alexey Navalny and other political prisoners in Russia,EP text adopted,9,2023-02-16,P9_TA(2023)0051,English,https://data.europarl.europa.eu/distribution/reds_iPlTa_Itm/TA-9-2023-0051/TA-9-2023-0051-FNL_en.pdf,https://data.europarl.europa.eu/distribution/reds_iPlTa_Itm/TA-9-2023-0051/TA-9-2023-0051-FNL_en.docx,NaN,NaN,http://publications.europa.eu/resource/authority/subject-matter/PESC; http://publications.europa.eu/resource/authority/subject-matter/DDLH,http://publications.europa.eu/resource/authority/dir-eu-legal-act/18,https://data.europarl.europa.eu/eli/dl/doc/B-9-2023-0133; https://data.europarl.europa.eu/eli/dl/doc/RC-9-2023-0133; https://data.europarl.europa.eu/eli/dl/doc/B-9-2023-0143; https://data.europarl...,https://data.europarl.europa.eu/eli/dl/proc/2023-2553,PE741.462,https://data.europarl.europa.eu/eli/dl/doc/TA-9-2023-0051,the recent deterioration of the inhuman imprisonment conditions of alexey navalny and other political prisoners in russia
39,TA-9-2023-0015,The establishment of a tribunal on the crime of aggression against Ukraine,EP text adopted,9,2023-01-19,P9_TA(2023)0015,English,https://data.europarl.europa.eu/distribution/reds_iPlTa_Itm/TA-9-2023-0015/TA-9-2023-0015-FNL_en.pdf,https://data.europarl.europa.eu/distribution/reds_iPlTa_Itm/TA-9-2023-0015/TA-9-2023-0015-FNL_en.docx,NaN,NaN,http://publications.europa.eu/resource/authority/subject-matter/PESC,http://publications.europa.eu/resource/authority/dir-eu-legal-act/18,https://data.europarl.europa.eu/eli/dl/doc/RC-9-2023-0063; https://data.europarl.europa.eu/eli/dl/doc/B-9-2023-0063; https://data.europarl.europa.eu/eli/dl/doc/B-9-2023-0069; https://data.europarl...,https://data.europarl.europa.eu/eli/dl/proc/2022-3017,PE741.383,https://data.europarl.europa.eu/eli/dl/doc/TA-9-2023-0015,the establishment of a tribunal on the crime of aggression against ukraine
97,TA-9-2023-0056,One year of Russia’s invasion and war of aggression against Ukraine,EP text adopted,9,2023-02-16,P9_TA(2023)0056,English,https://data.europarl.europa.eu/distribution/reds_iPlTa_Itm/TA-9-2023-0056/TA-9-2023-0056-FNL_en.pdf,https://data.europarl.europa.eu/distribution/reds_iPlTa_Itm/TA-9-2023-0056/TA-9-2023-0056-FNL_en.docx,NaN,NaN,http://publications.europa.eu/resource/authority/subject-matter/PESC,http://publications.europa.eu/resource/authority/dir-eu-legal-act/18,https://data.europarl.europa.eu/eli/dl/doc/RC-9-2023-0126; https://data.europarl.europa.eu/eli/dl/doc/B-9-2023-0139; https://data.europarl.europa.eu/eli/dl/doc/B-9-2023-0126; https://data.europarl...,https://data.europarl.europa.eu/eli/dl/proc/2023-2558,PE741.455,https://data.europarl.europa.eu/eli/dl/doc/TA-9-2023-0056,one year of russias invasion and war of aggression against ukraine
183,TA-9-2023-0115,"Repression in Russia, in particular the cases of Vladimir Kara-Murza and Aleksei Navalny",EP text adopted,9,2023-04-20,P9_TA(2023)0115,English,https://data.europarl.europa.eu/distribution/reds_iPlTa_Itm/TA-9-2023-0115/TA-9-2023-0115-FNL_en.pdf,https://data.europarl.europa.eu/distribution/reds_iPlTa_Itm/TA-9-2023-0115/TA-9-2023-0115-FNL_en.docx,NaN,NaN,http://publications.europa.eu/resource/authority/subject-matter/PESC; http://publications.europa.eu/resource/authority/subject-matter/DDLH,http://publications.europa.eu/resource/authority/dir-eu-legal-act/18,https://data.europarl.europa.eu/eli/dl/doc/B-9-2023-0226; https://data.europarl.europa.eu/eli/dl/doc/B-9-2023-0225; https://data.europarl.europa.eu/eli/dl/doc/B-9-2023-0229; https://data.europarl....,https://data.europarl.europa.eu/eli/dl/proc/

In [29]:
def process_link(link, output_folder, idx):
    """
    Downloads a Word document from a link, converts it to text, and saves it.
    Deletes the original .docx file after processing.

    Args:
        link (str): The URL to the Word document.
        output_folder (str): Path to the folder where files will be saved.
        idx (int): Index for naming the output files.

    Returns:
        str: A success or error message.
    """
    try:
        # Download the Word document
        response = requests.get(link)
        response.raise_for_status()  # Raise exception for HTTP errors
        word_file_path = os.path.join(output_folder, f"document_{idx + 1}.docx")
        
        # Save the Word file locally
        with open(word_file_path, 'wb') as word_file:
            word_file.write(response.content)
        
        # Convert Word document to text
        doc = Document(word_file_path)
        text = '\n'.join([paragraph.text for paragraph in doc.paragraphs])
        
        # Save the text to a file
        text_file_path = os.path.join(output_folder, f"document_{idx + 1}.txt")
        with open(text_file_path, 'w', encoding='utf-8') as text_file:
            text_file.write(text)
        
        # Delete the .docx file after processing
        os.remove(word_file_path)
        
        return f"Successfully processed: {link}"
    
    except Exception as e:
        return f"Error processing {link}: {e}"

def download_and_convert_docs_with_apply(links_series, output_folder):
    """
    Downloads Word documents from a pandas Series of links, converts them to text, 
    and saves the text files in the specified folder using `apply`.
    Deletes the original .docx files after processing.

    Args:
        links_series (pd.Series): A pandas Series containing URLs to Word documents.
        output_folder (str): Path to the folder where files will be saved.

    Returns:
        pd.Series: A Series of success/error messages for each link.
    """
    os.makedirs(output_folder, exist_ok=True)
    
    # Use apply with a lambda to pass arguments
    return links_series.apply(lambda link: process_link(link, output_folder, links_series[links_series == link].index[0]))

In [30]:
links_series = pd.Series(ds_filtered['document_doc'])
output_folder = "../data/processed/"

results = download_and_convert_docs_with_apply(links_series, output_folder)
print(results)


11     Successfully processed: https://data.europarl.europa.eu/distribution/reds_iPlTa_Itm/TA-9-2023-0051/TA-9-2023-0051-FNL_en.docx
39     Successfully processed: https://data.europarl.europa.eu/distribution/reds_iPlTa_Itm/TA-9-2023-0015/TA-9-2023-0015-FNL_en.docx
50     Successfully processed: https://data.europarl.europa.eu/distribution/reds_iPlTa_Itm/TA-9-2023-0029/TA-9-2023-0029-FNL_en.docx
97     Successfully processed: https://data.europarl.europa.eu/distribution/reds_iPlTa_Itm/TA-9-2023-0056/TA-9-2023-0056-FNL_en.docx
183    Successfully processed: https://data.europarl.europa.eu/distribution/reds_iPlTa_Itm/TA-9-2023-0115/TA-9-2023-0115-FNL_en.docx
210    Successfully processed: https://data.europarl.europa.eu/distribution/reds_iPlTa_Itm/TA-9-2023-0247/TA-9-2023-0247-FNL_en.docx
222    Successfully processed: https://data.europarl.europa.eu/distribution/reds_iPlTa_Itm/TA-9-2023-0283/TA-9-2023-0283-FNL_en.docx
233    Successfully processed: https://data.europarl.europa.eu/distri

In [98]:
# Define the directory paths
processed_dir = Path("../data/processed")
cleaned_dir = Path("../data/cleaned")

# Create the cleaned directory if it doesn't exist
cleaned_dir.mkdir(parents=True, exist_ok=True)

In [99]:
def clean_text(text):
    """
    Cleans and normalizes the input text.
    - Removes the 'TEXTS ADOPTED' header and its associated line.
    - Removes non-essential metadata like resolution codes.
    - Normalizes spaces and line breaks.
    """
    # Explicitly remove the "TEXTS ADOPTED" line
    text = re.sub(r"(?i)^TEXTS ADOPTED\s*\n", "", text)
    # Remove metadata patterns (example: P9_TA(2023)0051)
    text = re.sub(r"P\d+_TA\(\d{4}\)\d{4}", "", text)
    # Remove codes like PE741.462
    text = re.sub(r"PE\d+\.\d+", "", text)
    # Remove any residual leading/trailing spaces
    text = text.strip()
    # Replace multiple line breaks with a single one
    text = re.sub(r"\n\s*\n", "\n", text)
    # Normalize spaces
    text = re.sub(r"[ \t]+", " ", text)
    return text

In [100]:
def process_text_files(input_dir, output_dir):
    """
    Cleans and normalizes all text files in the input directory and saves them in the output directory.
    """
    for file_path in input_dir.glob("*.txt"):
        # Read the text content
        with open(file_path, "r", encoding="utf-8") as file:
            text = file.read()
        
        # Clean the text
        cleaned_text = clean_text(text)
        
        # Save the cleaned text
        output_path = output_dir / file_path.name
        with open(output_path, "w", encoding="utf-8") as file:
            file.write(cleaned_text)
        print(f"Processed and saved: {output_path}")

In [102]:
# Process all files in the 'processed' directory
process_text_files(processed_dir, cleaned_dir)

Processed and saved: ../data/cleaned/document_67.txt
Processed and saved: ../data/cleaned/document_98.txt
Processed and saved: ../data/cleaned/document_799.txt
Processed and saved: ../data/cleaned/document_348.txt
Processed and saved: ../data/cleaned/document_604.txt
Processed and saved: ../data/cleaned/document_65.txt
Processed and saved: ../data/cleaned/document_70.txt
Processed and saved: ../data/cleaned/document_417.txt
Processed and saved: ../data/cleaned/document_761.txt
Processed and saved: ../data/cleaned/document_211.txt
Processed and saved: ../data/cleaned/document_402.txt
Processed and saved: ../data/cleaned/document_158.txt
Processed and saved: ../data/cleaned/document_880.txt
Processed and saved: ../data/cleaned/document_117.txt
Processed and saved: ../data/cleaned/document_471.txt
Processed and saved: ../data/cleaned/document_673.txt
Processed and saved: ../data/cleaned/document_12.txt
Processed and saved: ../data/cleaned/document_460.txt
Processed and saved: ../data/clea